<h1 align=center><font size=6>IBM Applied Data Science Capstone Course by Coursera</font></h1>

# Week 5 Final Report

# Opening a New Restaurant in Mumbai,India

### *Build a dataframe of neighborhoods with coordinates in Mumbai,India by web scraping the data from Wikipedia page.
### *Obtain the venue data for the neighborhoods from Foursquare API.
### *Explore and cluster the neighborhoods.
### *Select the best cluster to open a new Restaurant.


## 1.Import library:

In [3]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [2]:
!pip install folium==0.5.0

     |████████████████████████████████| 81kB 7.2MB/s eta 0:00:011
  Stored in directory: /home/dsxuser/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


Libraries imported.


## 2. Scrap data from Wikipedia page into a DataFrame

In [4]:
neighbourhood=pd.read_html('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai')

In [5]:
neigh=neighbourhood[0]
neigh.head()

,Area,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270


In [6]:
neigh.shape

(93, 4)

In [7]:
neigh=neigh.drop(['Location'],axis=1)

In [8]:
neigh.head()

,Area,Latitude,Longitude
0,Amboli,19.129300,72.843400
1,"Chakala, Andheri",19.111388,72.860833
2,D.N. Nagar,19.124085,72.831373
3,Four Bungalows,19.124714,72.827210
4,Lokhandwala,19.130815,72.829270


In [9]:
neigh.rename(columns={"Area":"Neighbourhood"},inplace=True)
neigh.head()

,Neighbourhood,Latitude,Longitude
0,Amboli,19.129300,72.843400
1,"Chakala, Andheri",19.111388,72.860833
2,D.N. Nagar,19.124085,72.831373
3,Four Bungalows,19.124714,72.827210
4,Lokhandwala,19.130815,72.829270


In [10]:
neigh.to_csv('Neigh.csv',index=False)

## 3. Create a map of Mumbai, India with neighborhoods superimposed on top¶

In [11]:
# get the coordinates of Mumbai,india
address='Mumbai, India'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Mumbai, India 19.0759899, 72.8773928.


In [12]:
# create map of Mumbai using latitude and longitude values
map_mum = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(neigh['Latitude'], neigh['Longitude'], neigh['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  
    
map_mum

In [13]:
map_mum.save('map_mum.html')

## 4. Use the Foursquare API to explore the neighborhoods

In [1]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: 04FMIYLWCI0JRCJCZ2IA5CKQOMX3NDNOLGQLDXONFOJURNOV
CLIENT_SECRET:BE5F3J0TDWNKTKJ1NWXUH5BNBOU3ACWV4UEHWOHOGT2BSU53


In [15]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(neigh['Latitude'], neigh['Longitude'], neigh['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [16]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()


(7047, 7)


,Neighbourhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Amboli,19.1293,72.8434,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
1,Amboli,19.1293,72.8434,Merwans Cake shop,19.119300,72.845418,Bakery
2,Amboli,19.1293,72.8434,Jaffer Bhai's Delhi Darbar,19.137714,72.845909,Mughlai Restaurant
3,Amboli,19.1293,72.8434,Hard Rock Cafe Andheri,19.135995,72.835335,American Restaurant
4,Amboli,19.1293,72.8434,Pizza Express,19.131893,72.834668,Pizza Place


 ### Let's check how many venues were returned for each neighorhood

In [17]:
venues_df.groupby(["Neighbourhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhood,,,,,,
Aarey Milk Colony,17,17,17,17,17,17
Agripada,100,100,100,100,100,100
Altamount Road,99,99,99,99,99,99
Amboli,91,91,91,91,91,91
Amrut Nagar,100,100,100,100,100,100
Asalfa,100,100,100,100,100,100
Ballard Estate,97,97,97,97,97,97
Bandstand Promenade,77,77,77,77,77,77
Bangur Nagar,100,100,100,100,100,100


### Let's find out how many unique categories can be curated from all the returned venues

In [18]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 217 uniques categories.


In [19]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Falafel Restaurant', 'Bakery', 'Mughlai Restaurant',
       'American Restaurant', 'Pizza Place', 'Brewery',
       'Chinese Restaurant', 'Pub', 'Multiplex', 'Ice Cream Shop', 'Café',
       'Mediterranean Restaurant', 'Indian Restaurant', 'BBQ Joint',
       'Sandwich Place', 'Lounge', 'Diner', 'Coffee Shop', 'Juice Bar',
       'Spa', "Women's Store", 'Jewelry Store', 'Fast Food Restaurant',
       'Gym', 'Comfort Food Restaurant', 'Bar', 'Movie Theater',
       'Dessert Shop', 'Residential Building (Apartment / Condo)',
       'Italian Restaurant', 'Deli / Bodega', 'Asian Restaurant',
       'Seafood Restaurant', 'Building', 'Snack Place', 'Fish Market',
       'Vegetarian / Vegan Restaurant', 'Clothing Store',
       'Department Store', 'Hotel', 'Camera Store', 'Athletics & Sports',
       'Shopping Mall', 'Market', 'Restaurant',
       'Maharashtrian Restaurant', 'Food Truck', 'Airport Service',
       'Nightclub', 'Cocktail Bar'], dtype=object)

In [20]:
# check if the results contain "Restaurant"
"Restaurant" in venues_df['VenueCategory'].unique()

True

### 5. Analyze Each Neighborhood¶

In [21]:
# one hot encoding
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
mum_onehot['Neighbourhoods'] = venues_df['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(7047, 218)


,Neighbourhoods,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Water Park,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Amboli,0,0,

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
mum_grouped = mum_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped

(92, 218)


,Neighbourhoods,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Water Park,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.058824,0.000000,0.00,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.058824,0.00,0.000000,0.058824,0.00,0.058824,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.058824,0.05882

In [23]:

len(mum_grouped[mum_grouped["Restaurant"] > 0])

77

### Create a new DataFrame for Shopping Mall data only

In [24]:
mum_rest = mum_grouped[["Neighbourhoods","Restaurant"]]

In [25]:
mum_rest.head()

,Neighbourhoods,Restaurant
0,Aarey Milk Colony,0.058824
1,Agripada,0.060000
2,Altamount Road,0.050505
3,Amboli,0.000000
4,Amrut Nagar,0.050000


### 6. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.



In [33]:
# set number of clusters
kclusters = 3

mum_clustering = mum_rest.drop(["Neighbourhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 0, 2, 2, 0, 0, 0, 0], dtype=int32)

In [34]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mum_merged = mum_rest.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_


In [35]:
mum_merged.rename(columns={"Neighbourhoods": "Neighbourhood"}, inplace=True)
mum_merged.head()

,Neighbourhood,Restaurant,Cluster Labels
0,Aarey Milk Colony,0.058824,2
1,Agripada,0.060000,2
2,Altamount Road,0.050505,2
3,Amboli,0.000000,0
4,Amrut Nagar,0.050000,2


In [36]:
# merge mumbai_grouped with mumbai_data to add latitude/longitude for each neighborhood
mum_merged = mum_merged.join(neigh.set_index("Neighbourhood"), on="Neighbourhood")

print(mum_merged.shape)
mum_merged.head() # check the last columns!


(92, 5)


,Neighbourhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.058824,2,19.148493,72.881756
1,Agripada,0.060000,2,18.977700,72.827300
2,Altamount Road,0.050505,2,18.968100,72.809500
3,Amboli,0.000000,0,19.129300,72.843400
4,Amrut Nagar,0.050000,2,19.102077,72.912835


In [37]:
# sort the results by Cluster Labels
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged


(92, 5)


,Neighbourhood,Restaurant,Cluster Labels,Latitude,Longitude
45,Juhu,0.010000,0,19.100000,72.830000
31,Deonar,0.013158,0,19.050000,72.890000
32,Dharavi,0.010000,0,19.040208,72.850850
34,Dindoshi,0.000000,0,19.176382,72.864891
73,Pali Naka,0.020000,0,19.062742,72.829396
72,Pali Hill,0.010000,0,19.068000,72.826000
37,Four Bungalows,0.000000,0,19.124714,72.827210
38,Gorai,0.000000,0,19.250057,72.782021
71,Nehru Nagar,0.000000,0,15.451686,74.971977
40,Hiranandani Gardens,0.024691,0,19.118986,72.911767


### Finally, let's visualize the resulting clusters

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighbourhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 7. Examine Clusters

### Cluster 0

In [42]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighbourhood,Restaurant,Cluster Labels,Latitude,Longitude
45,Juhu,0.010000,0,19.100000,72.830000
31,Deonar,0.013158,0,19.050000,72.890000
32,Dharavi,0.010000,0,19.040208,72.850850
34,Dindoshi,0.000000,0,19.176382,72.864891
73,Pali Naka,0.020000,0,19.062742,72.829396
72,Pali Hill,0.010000,0,19.068000,72.826000
37,Four Bungalows,0.000000,0,19.124714,72.827210
38,Gorai,0.000000,0,19.250057,72.782021
71,Nehru Nagar,0.000000,0,15.451686,74.971977
40,Hiranandani Gardens,0.024691,0,19.118986,72.911767


### Cluster 1

In [46]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighbourhood,Restaurant,Cluster Labels,Latitude,Longitude
84,Uttan,0.4,1,19.28,72.785


### Cluster 2

In [44]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighbourhood,Restaurant,Cluster Labels,Latitude,Longitude
89,Virar,0.105263,2,19.470000,72.800000
86,Vidyavihar,0.050000,2,19.080000,72.896000
75,Parel,0.040000,2,18.990000,72.840000
66,Nahur,0.040000,2,19.157000,72.941000
74,Pant Nagar,0.050000,2,19.080000,72.910000
78,Sahar,0.040000,2,19.098889,72.867222
77,Prabhadevi,0.040000,2,19.016600,72.829500
65,Mumbai Central,0.060000,2,18.969700,72.819400
68,Nalasopara,0.142857,2,19.415400,72.861300
0,Aarey Milk Colony,0.058824,2,19.148493,72.881756


## Observations-

By using K-mean clustering technique we being able to cluster the neighbourhoods of Mumbai to 3 categories .The neighbourhoods falling under cluster 0 have very few restaurant which is a great oppurtunity for investors to open a new retaurant business.The neighbourhoods falling under cluster 1 have very large restaurants so one couldn't want to take that much risk as already cut-throat compition is present in this area and the neighbourhoods falling under cluster 2 also have moderate competion ,so investors are advised to avoid cluster 1('UTTAN') for their location selection.And similary cluster 0 will more suitable for them as as compared to cluster 2 as to avoid standard competion.